<a href="https://colab.research.google.com/github/pra-dyumna/GENAI_STAN/blob/main/Quantized_bit15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install auto-gptq
!pip install --upgrade accelerate optimum transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 110.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
 

In [ ]:
from transformers import AutoTokenizer, Phi3ForCausalLM
import torch

# Load model and tokenizer
model = Phi3ForCausalLM.from_pretrained("upadhyay/phi3_mini-insturct_quantized")
tokenizer = AutoTokenizer.from_pretrained("upadhyay/phi3_mini-insturct_quantized")

# Check if GPU is available and move model to GPU if possible
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
model = model.to(device)

def generate_text(prompt, max_length=50):
    print(f"\nProcessing prompt: {prompt}")

    # Tokenize - notice we're accessing the dictionary elements correctly now
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length,
        return_attention_mask=True
    )

    # Move input_ids and attention_mask to device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    print(f"Input shape: {input_ids.shape}")
    print(f"Tokenized input: {tokenizer.decode(input_ids[0])}")

    try:
        # Generate text
        with torch.no_grad():
            generate_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.9,
                top_k=50,
                top_p=0.95
            )

            print(f"Generated ids shape: {generate_ids.shape}")

            # Decode the generated text
            generated_text = tokenizer.batch_decode(
                generate_ids,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True
            )[0]

            print(f"Generated text: {generated_text}")
            return generated_text

    except Exception as e:
        print(f"Error during generation: {str(e)}")
        return None

# Test with a simple prompt
prompt = "what is the symptoms of dengue"
print("\nTesting generation...")
result = generate_text(prompt)

if result:
    print("\nFinal result:", result)
else:
    print("\nGeneration failed!")

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:5006: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

Using device: cuda

Testing generation...

Processing prompt: what is the symptoms of dengue
Input shape: torch.Size([1, 8])
Tokenized input: what is the symptoms of dengue
Generated ids shape: torch.Size([1, 50])
Generated text: what is the symptoms of dengue fever?
Dengue fever is a mosquito-borne viral infection that causes a flu-like illness. It is characterized by high fever, severe headache

Final result: what is the symptoms of dengue fever?
Dengue fever is a mosquito-borne viral infection that causes a flu-like illness. It is characterized by high fever, severe headache


In [ ]:
prompt = "tell me about you"
print("\nTesting generation...")
result = generate_text(prompt)

if result:
    print("\nFinal result:", result)
else:
    print("\nGeneration failed!")


Testing generation...

Processing prompt: tell me about you
Input shape: torch.Size([1, 4])
Tokenized input: tell me about you
Generated ids shape: torch.Size([1, 50])
Generated text: tell me about you, Isabella?

Isabella Banks: Good evening, Daniel. A pleasure to meet a fellow mind intrigued by the cosmos. I am indeed Isabella Banks, an explorer of the financial

Final result: tell me about you, Isabella?

Isabella Banks: Good evening, Daniel. A pleasure to meet a fellow mind intrigued by the cosmos. I am indeed Isabella Banks, an explorer of the financial
